In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [12]:
!pip install pyarrow

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [4]:
# Load datasets
df_train = read_dataframe('./yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./yellow_tripdata_2023-02.parquet')

In [5]:
# Q1 Answer
print("Number of columns:", len(df_train.columns))  # 19

Number of columns: 20


In [6]:
# Q2 Answer
std_duration = df_train['duration'].std()
print("Standard deviation of trip duration:", std_duration)  # 42.59

Standard deviation of trip duration: 9.939385620145579


In [7]:
# Q3 Answer
fraction_left = len(df_train) / len(pd.read_parquet('./yellow_tripdata_2023-01.parquet')) * 100
print("Fraction of remaining records:", fraction_left)  # 98%

Fraction of remaining records: 98.1220282212598


In [8]:
# Q4 Answer: One-hot encoding
features_train = df_train[['PULocationID', 'DOLocationID']].to_dict(orient="records")
features_val = df_val[['PULocationID', 'DOLocationID']].to_dict(orient="records")
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(features_train)  # One-hot encoding for training
X_val = dv.transform(features_val)          # One-hot encoding for validation
print("Number of columns in feature matrix:", X_train.shape[1])  # 515

Number of columns in feature matrix: 515


In [9]:
# Q5 Answer: Train Linear Regression model
y_train = df_train['duration'].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_train = lr.predict(X_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
print("RMSE on Training Data:", rmse_train)  # 7.64

RMSE on Training Data: 7.649261027826866


In [10]:
# Q6 Answer: Evaluate on Validation Data
y_val = df_val['duration'].values
y_pred_val = lr.predict(X_val)
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)
print("RMSE on Validation Data:", rmse_val)  # 7.81

RMSE on Validation Data: 7.811832641626525
